### Problem statement: How good is a player X against CSK in Chepauk when they're playing 3 spinners?

### Filters in Problem statement

- Venue - 'MA Chidambaram Stadium, Chepauk'
- Phase - 'Middle'
- Opponent - 'Chennai Super Kings'

In [7]:
# Importing all warnings & libraries

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [8]:
# Importing the 'deliveries' and 'matches' data

deliveries = pd.read_csv('deliveries.csv')
matches = pd.read_csv('matches.csv')

In [9]:
# Function for Balls per Dismissals

def balls_per_dismissal(balls, dismissals):
    if dismissals > 0:
        return balls/dismissals
    else:
        return balls/1

In [10]:
# Function for Balls per Boundary

def balls_per_boundary(balls, boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls/1

In [11]:
# Function defined using the Dataframe with filters like venue, phase & opposition

def ByCustom(df, current_venue, current_phase, current_opposition):
    df = df[df.venue == current_venue]
    df = df[df.phase == current_phase]
    df = df[df.bowling_team == current_opposition]
    df.reset_index(inplace = True, drop = True)
    
    df['isDot'] = df['batsman_runs'].apply(lambda x: 1 if x == 0 else 0)
    df['isOne'] = df['batsman_runs'].apply(lambda x: 1 if x == 1 else 0)
    df['isTwo'] = df['batsman_runs'].apply(lambda x: 1 if x == 2 else 0)
    df['isThree'] = df['batsman_runs'].apply(lambda x: 1 if x == 3 else 0)
    
    df['isFour'] = df['batsman_runs'].apply(lambda x: 1 if x == 4 else 0)
    df['isSix'] = df['batsman_runs'].apply(lambda x: 1 if x == 6 else 0)
    
    runs = pd.DataFrame(df.groupby(['batsman'])['batsman_runs'].sum()).reset_index().rename(columns = {'batsman_runs':'runs'})
    balls = pd.DataFrame(df.groupby(['batsman'])['match_id'].count()).reset_index().rename(columns = {'match_id':'balls'})
    innings = pd.DataFrame(df.groupby(['batsman'])['match_id'].apply(lambda x: len(list(np.unique(x))))).reset_index().rename(columns = {'match_id':'innings'})
    dismissals = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns = {'player_dismissed':'dismissals'})
    sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns = {'isSix':'sixes'})
    fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns = {'isFour':'fours'})
    
    dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum()).reset_index().rename(columns = {'isDot':'dots'})
    
    df = pd.merge(innings, runs, on = 'batsman').merge(balls, on = 'batsman').merge(dismissals, on = 'batsman').merge(fours, on = 'batsman').merge(sixes, on = 'batsman').merge(dots, on = 'batsman')
    df['RPI'] = df.apply(lambda x: (x['runs']/x['innings']), axis=1)
    df['SR'] = df.apply(lambda x: 100*(x['runs']/x['balls']), axis=1)
    df['BPD'] = df.apply(lambda x: balls_per_dismissal(x['balls'], x['dismissals']), axis=1)
    df['BPB'] = df.apply(lambda x: balls_per_boundary(x['balls'], (x['fours'] + x['sixes'])), axis=1)
    
    df['dot_percentage'] = df.apply(lambda x: (x['dots']/x['balls']), axis=1)
    
    return df

In [12]:
# Copyig the deliveries dataset into Dataframe df for analyzing.

df = deliveries.copy()

In [13]:
# Copyig the matches dataset into Dataframe mdf for analyzing.

mdf = matches.copy()

In [14]:
# Renaming column 'id' to 'match_id' for easy joining.

mdf = mdf.rename(columns = {'id':'match_id'})

In [15]:
# Merging df & mdf to form a combined dataframe 'comb'.

comb = pd.merge(df, mdf, on = 'match_id', how = 'left')

In [16]:
comb['over_no'] = comb['over'].apply(lambda x: x - 1)

In [17]:
# Function definition to get various phases in a match.

def get_phase(over_no):
    if over_no < 6:
        return 'Powerplay'
    elif over_no < 15:
        return 'Middle'
    else:
        return 'Death'

In [18]:
comb['phase'] = comb['over_no'].apply(lambda x: get_phase(x))

In [19]:
# Dataframe to get the data corresponding to our initial filters considered.

df = ByCustom(comb, 'MA Chidambaram Stadium, Chepauk', 'Middle', 'Chennai Super Kings')

In [20]:
df.head(5)

,batsman,innings,runs,balls,dismissals,fours,sixes,dots,RPI,SR,BPD,BPB,dot_percentage
0,A Mishra,1,14,16,1,2,0,9,14.0,87.500000,16.0,8.000000,0.562500
1,A Mithun,1,11,8,1,2,0,3,11.0,137.500000,8.0,4.000000,0.375000
2,A Symonds,1,27,27,0,2,1,12,27.0,100.000000,27.0,9.000000,0.444444
3,AB Agarkar,1,6,2,0,1,0,0,6.0,300.000000,2.0,2.000000,0.000000
4,AB de Villiers,4,88,61,1,10,1,17,22.0,144.262295,61.0,5.545455,0.278689


In [21]:
# Assigning weights to various considered factors using MCDM method.

wt_sr, wt_rpi, wt_bpd, wt_dot_percentage = 0.13, 0.27, 0.16, 0.45

#### Filter to avoid outliers

In [22]:
df = df[(df.innings >= 2) & (df.balls >= 20)]

#### Calculation - Normalizing all values to same dimensions.

In [23]:
df['calc_SR'] = df['SR'].apply(lambda x: x*x)
df['calc_RPI'] = df['RPI'].apply(lambda x: x*x)
df['calc_BPD'] = df['BPD'].apply(lambda x: x*x)
df['calc_dot_percentage'] = df['dot_percentage'].apply(lambda x: x*x)

sq_sr, sq_rpi, sq_bpd, sq_dot_percentage = np.sqrt(df[['calc_SR', 'calc_RPI', 'calc_BPD', 'calc_dot_percentage']].sum(axis = 0))

df['calc_SR'] = df['calc_SR'].apply(lambda x: x/sq_sr)
df['calc_RPI'] = df['calc_RPI'].apply(lambda x: x/sq_rpi)
df['calc_BPD'] = df['calc_BPD'].apply(lambda x: x/sq_bpd)
df['calc_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: x/sq_dot_percentage)

df['calc_SR'] = df['calc_SR'].apply(lambda x: x*wt_sr)
df['calc_RPI'] = df['calc_RPI'].apply(lambda x: x*wt_rpi)
df['calc_BPD'] = df['calc_BPD'].apply(lambda x: x*wt_bpd)
df['calc_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: x*wt_dot_percentage)

best_sr, worst_sr = max(df['calc_SR']), min(df['calc_SR'])
best_rpi, worst_rpi = max(df['calc_RPI']), min(df['calc_RPI'])
best_bpd, worst_bpd = max(df['calc_BPD']), min(df['calc_BPD'])
best_dot_percentage, worst_dot_percentage = min(df['calc_dot_percentage']), max(df['calc_dot_percentage'])

#### Calculation - Comparison against the best & worst values.

In [24]:
df['dev_best_sr'] = df['calc_SR'].apply(lambda x: (x-best_sr)*(x-best_sr))
df['dev_best_rpi'] = df['calc_RPI'].apply(lambda x: (x-best_rpi)*(x-best_rpi))
df['dev_best_bpd'] = df['calc_BPD'].apply(lambda x: (x-best_bpd)*(x-best_bpd))
df['dev_best_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: (x-best_dot_percentage)*(x-best_dot_percentage))

df['dev_best_sqrt'] = df.apply(lambda x: x['dev_best_sr'] + x['dev_best_rpi'] + x['dev_best_bpd'] + x['dev_best_dot_percentage'], axis = 1)

df['dev_worst_sr'] = df['calc_SR'].apply(lambda x: (x-worst_sr)*(x-worst_sr))
df['dev_worst_rpi'] = df['calc_RPI'].apply(lambda x: (x-worst_rpi)*(x-worst_rpi))
df['dev_worst_bpd'] = df['calc_BPD'].apply(lambda x: (x-worst_bpd)*(x-worst_bpd))
df['dev_worst_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: (x-worst_dot_percentage)*(x-worst_dot_percentage))

df['dev_worst_sqrt'] = df.apply(lambda x: x['dev_worst_sr'] + x['dev_worst_rpi'] + x['dev_worst_bpd'] + x['dev_worst_dot_percentage'], axis = 1)

#### Overall Score for each player (values from 0 to 1)

In [25]:
df['score'] = df.apply(lambda x: x['dev_worst_sqrt']/(x['dev_worst_sqrt'] + x['dev_best_sqrt']), axis = 1)

In [26]:
df[['batsman', 'innings', 'runs', 'dismissals', 'SR', 'dots', 'RPI', 'score']].sort_values(by = 'score', ascending = False).reset_index(drop = True).head(10)

,batsman,innings,runs,dismissals,SR,dots,RPI,score
0,SE Marsh,3,104,1,192.592593,10,34.666667,0.948585
1,SR Watson,4,121,2,172.857143,16,30.250000,0.752836
2,AB de Villiers,4,88,1,144.262295,17,22.000000,0.621450
3,G Gambhir,3,86,1,117.808219,21,28.666667,0.554617
4,NV Ojha,3,64,2,156.097561,12,21.333333,0.483739
5,Y Venugopal Rao,3,67,1,131.372549,16,22.333333,0.407355
6,MS Bisla,3,89,2,134.848485,20,29.666667,0.401361
7,V Kohli,7,175,4,116.666667,50,25.000000,0.228748
8,DJ Bravo,2,26,2,130.000000,7,13.000000,0.206668
9,AD Mathews,2,42,0,116.666667,12,21.000000,0.193248


### Conclusion

From the above analysis for a Player 'X' we have the Top 10 batters who play spin well at Chepauk against CSK when CSK possess three spinners.

We could easily see that Shaun Marsh stands top of the ladder with a score of 0.94 with 104 runs in just 3 innings at a whopping strike rate of 192.6 approximately with a staggering Runs per Innings of 34.6 playing just 10 dots and dismissed once.

Marsh is followed by other IPL Legends in Shane Watson & AB de Villiers in the final ladder.